In [ ]:
import datetime as dt
import time
import random
import logging

from optibook.synchronous_client import Exchange

exchange = Exchange()
exchange.connect()

logging.getLogger('client').setLevel('ERROR')

def delta_limit_breached(volume,side,limit):
    positions = exchange.get_positions()
    position_A = positions['PHILIPS_A']
    position_B = positions['PHILIPS_B']
    if side == 'bid':
        return abs(position_A + position_B + volume) < limit
    elif side == 'ask':
        return abs(position_A + position_B - volume) < limit
    else:
        raise Exception(f'''Invalid side provided: {side}, expecting 'bid' or 'ask'.''')
    
def trade_would_breach_position_limit(instrument_id, volume, side, position_limit=10):
    positions = exchange.get_positions()
    position_instrument = positions[instrument_id]

    if side == 'bid':
        return position_instrument + volume > position_limit
    elif side == 'ask':
        return position_instrument - volume < -position_limit
    else:
        raise Exception(f'''Invalid side provided: {side}, expecting 'bid' or 'ask'.''')


def print_positions_and_pnl():
    positions = exchange.get_positions()
    pnl = exchange.get_pnl()

    print('Positions:')
    for instrument_id in positions:
        print(f'  {instrument_id:10s}: {positions[instrument_id]:4.0f}')

    print(f'\nPnL: {pnl:.2f}')


STOCK_A_ID = 'PHILIPS_A'
STOCK_B_ID = 'PHILIPS_B'

while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    print_positions_and_pnl()
    print(f'')

    # Flip a coin to select which stock to trade
    # if random.random() > 0.5:
    #     stock_id = STOCK_A_ID
    # else:
    #     stock_id = STOCK_B_ID
    # print(f'Selected stock {stock_id} to trade.')

    # Obtain order book and only trade if there are both bids and offers available on that stock
    
    stock_id = STOCK_A_ID
    
    tradeticks = exchange.poll_new_trade_ticks(stock_id)
    i=0
    avgderA=0
    for t in tradeticks:
        if(i==25):
            break
        elif(i==0):
            i=i+1
            continue
        avgderA=avgderA+(25-i)*(tradeticks[i].price-tradeticks[i-1].price)
        i=i+1
    avgderA=avgderA/5
    
    stock_id = STOCK_B_ID
    
    tradeticks = exchange.poll_new_trade_ticks(stock_id)
    i=0
    avgderB=0
    for t in tradeticks:
        if(i==25):
            break
        elif(i==0):
            i=i+1
            continue
        avgderB=avgderB+(25-i)*(tradeticks[i].price-tradeticks[i-1].price)
        i=i+1
    avgderB=avgderB/5
    
    print(avgderA)
    print(avgderB)
    if(avgderA>0.1):
        stock_id = STOCK_A_ID
        side='bid'
    elif(avgderA<0):
        if random.random() > 0.25:
            side = 'ask'
            stock_id = STOCK_A_ID
        else:
            side = 'bid'
            stock_id = STOCK_B_ID
    elif(avgderB<0):
        stock_id = STOCK_B_ID
        side='ask'
    else:
        stock_id = STOCK_B_ID
        side='bid'
    
    stock_order_book = exchange.get_last_price_book(stock_id)
    if not (stock_order_book and stock_order_book.bids and stock_order_book.asks):
        print(f'Order book for {stock_id} does not have bids or offers. Skipping iteration.')
        continue

    # Obtain best bid and ask prices from order book
    best_bid_price = stock_order_book.bids[0].price
    best_ask_price = stock_order_book.asks[0].price
    print(f'Top level prices for {stock_id}: {best_bid_price:.2f} :: {best_ask_price:.2f}')
    
    # Flip a coin to decide whether to buy or sell
    if side == 'bid':
        price = best_ask_price
    else:
        price = best_bid_price

    # Insert an IOC order to trade the opposing top-level
    volume = 1
    if not delta_limit_breached(volume,side,10):
        time.sleep(30)
        continue
    if not trade_would_breach_position_limit(stock_id, volume, side):
        print(f'''Inserting {side} for {stock_id}: {volume:.0f} lot(s) at price {price:.2f}.''')
        exchange.insert_order(
            instrument_id=stock_id,
            price=price,
            volume=volume,
            side=side,
            order_type='ioc')
    else:
        print(f'''Not inserting {volume:.0f} lot {side} for {stock_id} to avoid position-limit breach.''')

    print(f'\nSleeping for 5 seconds.')
    time.sleep(30)


2022-01-23 12:07:58,701 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2022-01-23 12:07:58,827 [client    ] [Thread-18   ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.



-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2022-01-23 12:07:58.822798 UTC.
-----------------------------------------------------------------
Positions:
  PHILIPS_A :    1
  PHILIPS_B :    8

PnL: -55.10

0.0
0.0
Top level prices for PHILIPS_B: 100.80 :: 101.10

-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2022-01-23 12:08:28.824412 UTC.
-----------------------------------------------------------------
Positions:
  PHILIPS_A :    1
  PHILIPS_B :    8

PnL: -60.10

-1.3199999999999874
0.0
Top level prices for PHILIPS_A: 100.60 :: 101.30
Inserting ask for PHILIPS_A: 1 lot(s) at price 100.60.

Sleeping for 5 seconds.

-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2022-01-23 12:08:58.866118 UTC.
-----------------------------------------------------------------
Positions:
  PHILIPS_A :    0
  PHILIPS_B :    8

PnL: -57.00

2.31999999